In [ ]:
#Download file from link https://www.cse.wustl.edu/~jain/ehms/ftp/wustl-ehms-2020_with_attacks_categories.csv

!wget https://www.cse.wustl.edu/~jain/ehms/ftp/wustl-ehms-2020_with_attacks_categories.csv

!pip install xgboost numpy scikit-learn adversarial-robustness-toolbox


--2025-02-01 21:37:56--  https://www.cse.wustl.edu/~jain/ehms/ftp/wustl-ehms-2020_with_attacks_categories.csv
Resolving www.cse.wustl.edu (www.cse.wustl.edu)... 128.252.167.136
Connecting to www.cse.wustl.edu (www.cse.wustl.edu)|128.252.167.136|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3946888 (3.8M) [text/csv]
Saving to: ‘wustl-ehms-2020_with_attacks_categories.csv.1’

wustl-ehms-2020_wit 100%[===================>]   3.76M  13.2MB/s    in 0.3s    

2025-02-01 21:37:57 (13.2 MB/s) - ‘wustl-ehms-2020_with_attacks_categories.csv.1’ saved [3946888/3946888]



In [ ]:
!pip install xgboost numpy scikit-learn adversarial-robustness-toolbox

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 17.8 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
from sklearn.svm import SVC
from art.estimators.classification import SklearnClassifier
from sklearn.preprocessing import LabelEncoder, StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score


# Load WUSTL-EHMS-2020 Dataset
df = pd.read_csv('wustl-ehms-2020_with_attacks_categories.csv')

# Drop unwanted columns
col_ignore = ['Flgs', 'Dir', 'Label', 'Packet_num']
col_address = ['DstMac', 'SrcMac', 'SrcAddr', 'DstAddr', 'Sport']
col_biomedic = ['Temp', 'SpO2', 'Pulse_Rate', 'SYS', 'DIA', 'Heart_rate', 'Resp_Rate', 'ST']
df = df.drop(columns=col_ignore + col_address + col_biomedic)

# Encode labels
label_encoder = LabelEncoder()
df['Attack Category'] = label_encoder.fit_transform(df['Attack Category'])

# Separate features and target
X = df.drop(columns=['Attack Category'])
y = df['Attack Category']

# One-hot encode the target labels
one_hot_encoder = OneHotEncoder(sparse_output=False)
y = one_hot_encoder.fit_transform(y.values.reshape(-1, 1))

# Split dataset into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalize features for better numerical stability
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Define clip values for ART
clip_values = (X_train.min(), X_train.max())

# Step 2: Create the model
model = SVC(C=1.0, kernel="rbf", probability=True)  # Enable probability for ART compatibility
#model_etc_2 = SVC(C=1.0, kernel="rbf", probability=True)  # Enable probability for ART compatibility

# Step 3: Create the ART classifier
classifier = SklearnClassifier(model=model, clip_values=clip_values)

# Step 4: Train the ART classifier
classifier.fit(X_train, y_train)

# Step 5: Evaluate the ART classifier on benign test examples
predictions = classifier.predict(X_test)
accuracy = accuracy_score(np.argmax(y_test, axis=1), np.argmax(predictions, axis=1))
print("Accuracy on benign test examples: {}%".format(accuracy * 100))

Accuracy on benign test examples: 92.86151960784314%


In [ ]:
from art.attacks.evasion import FastGradientMethod

# Step 6: Generate adversarial test examples
attack = FastGradientMethod(estimator=classifier, eps=0.5)  # Increase epsilon for stronger attack
x_test_adv = attack.generate(x=X_test)

# Step 7: Evaluate the ART classifier on adversarial test examples
predictions = classifier.predict(x_test_adv)
accuracy = np.sum(np.argmax(predictions, axis=1) == np.argmax(y_test, axis=1)) / len(y_test)
print("Accuracy on adversarial test examples: {}%".format(accuracy * 100))

# Adfter attack 87%

Accuracy on adversarial test examples: 87.86764705882352%


In [ ]:
from sklearn.metrics import f1_score
from art.attacks.evasion import SaliencyMapMethod

# Attack JSMA: Generate adversarial test examples using JSMA
attack_jsma = SaliencyMapMethod(classifier=classifier, theta=0.1, gamma=0.5)
x_test_jsma = attack_jsma.generate(x=X_test)

# Attack JSMA: Evaluate the ART classifier on adversarial test examples
predictions = classifier.predict(x_test_jsma)
accuracy = np.sum(np.argmax(predictions, axis=1) == np.argmax(y_test, axis=1)) / len(y_test)
f1 = f1_score(np.argmax(y_test, axis=1), np.argmax(predictions, axis=1), average='weighted')  # Calculate F1 score
print("Accuracy on adversarial test examples (JSMA): {:.2f}%".format(accuracy * 100))
print("F1 score on adversarial test examples (JSMA): {:.4f}".format(f1))

JSMA:   0%|          | 0/3264 [00:00<?, ?it/s]

Accuracy on adversarial test examples (JSMA): 87.25%
F1 score on adversarial test examples (JSMA): 0.8132


In [ ]:
# # Attack JSMA: Evaluate the ART classifier on adversarial test examples
# predictions = classifier.predict(x_test_jsma)
# accuracy = np.sum(np.argmax(predictions, axis=1) == np.argmax(y_test, axis=1)) / len(y_test)
# f1 = f1_score(np.argmax(y_test, axis=1), np.argmax(predictions, axis=1), average='weighted')  # Calculate F1 score
# print("Accuracy on adversarial test examples (JSMA): {:.2f}%".format(accuracy * 100))
# print("F1 score on adversarial test examples (JSMA): {:.4f}".format(f1))

In [ ]:
# Example: Generate adversarial examples using DeepFool
from art.attacks.evasion import DeepFool
attack_deepfool = DeepFool(classifier=classifier)
x_adv_deepfool = attack_deepfool.generate(x=X_test)

predictions = classifier.predict(x_adv_deepfool)
accuracy = np.sum(np.argmax(predictions, axis=1) == np.argmax(y_test, axis=1)) / len(y_test)
f1 = f1_score(np.argmax(y_test, axis=1), np.argmax(predictions, axis=1), average='weighted')  # Calculate F1 score
print("Accuracy on adversarial test examples (JSMA): {:.2f}%".format(accuracy * 100))
print("F1 score on adversarial test examples (JSMA): {:.4f}".format(f1))

DeepFool:   0%|          | 0/3264 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
from sklearn.metrics import f1_score
from art.attacks.evasion import CarliniL2Method  # Import Carlini L2 attack

# Example: Generate adversarial examples using Carlini & Wagner L2 attack
attack_cw = CarliniL2Method(classifier=classifier)  # Initialize Carlini L2 attack
x_adv_cw = attack_cw.generate(x=X_test)  # Generate adversarial examples

# Evaluate model on adversarial examples
predictions = classifier.predict(x_adv_cw)

# Calculate accuracy
accuracy = np.sum(np.argmax(predictions, axis=1) == np.argmax(y_test, axis=1)) / len(y_test)

# Calculate F1 score
f1 = f1_score(np.argmax(y_test, axis=1), np.argmax(predictions, axis=1), average='weighted')

# Print results
print("Accuracy on adversarial test examples (Carlini L2): {:.2f}%".format(accuracy * 100))
print("F1 score on adversarial test examples (Carlini L2): {:.4f}".format(f1))

C&W L_2:   0%|          | 0/3264 [00:00<?, ?it/s]

In [ ]:
# # Other attacks:

# https://github.com/Trusted-AI/adversarial-robustness-toolbox/tree/main/art/attacks

# # Example: Create a target model (e.g., SVM)
# model = SVC(C=1.0, kernel="rbf")
# classifier = SklearnClassifier(model=model, clip_values=(0, 1))

# Example: Generate adversarial examples using FGSM
attack_fgsm = FastGradientMethod(estimator=classifier, eps=0.1)
x_adv_fgsm = attack_fgsm.generate(x=x_test)

# Example: Generate adversarial examples using JSMA
attack_jsma = SaliencyMapMethod(classifier=classifier, theta=0.1, gamma=0.1)
x_adv_jsma = attack_jsma.generate(x=x_test)

# Example: Generate adversarial examples using DeepFool
attack_deepfool = DeepFool(classifier=classifier)
x_adv_deepfool = attack_deepfool.generate(x=x_test)

# Example: Generate adversarial examples using C & W Attack
attack_cw = CarliniL2Method(classifier=classifier)
x_adv_cw = attack_cw.generate(x=x_test)

# Example: Generate adversarial examples using BIM
attack_bim = BasicIterativeMethod(estimator=classifier, eps=0.1, eps_step=0.01, max_iter=100)
x_adv_bim = attack_bim.generate(x=x_test)

# Example: Generate adversarial examples using PGD
attack_pgd = ProjectedGradientDescent(estimator=classifier, eps=0.1, eps_step=0.01, max_iter=100)
x_adv_pgd = attack_pgd.generate(x=x_test)

# Example: Generate adversarial examples using EAD
attack_ead = ElasticNet(classifier=classifier)
x_adv_ead = attack_ead.generate(x=x_test)